In [ ]:
import time
import matplotlib.pyplot as plt

## Numba

 You have some code... you want to make it run fast 

### Why do we use Python?

+ Readability and simplicity
+ Rapid development and protyping
+ Rich ecosystem of libraries and frameworks
+ Large and helpful community

### What are we missing?

+ Speed?

## Compiled code vs Interpreted Code

![](figures/program_life.png)

![](figures/program_life_python.png)

![](figures/program_life_compiler.png)

## How Numba works

In [14]:
import numpy as np
import numba
import time

In [5]:
def integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s

In [6]:
a = 0
b = 2
n = 1_000_000

In [7]:
plaintime = %timeit -o integrate(a, b, n)

137 ms ± 1.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
#Add Numba decorator here
@numba.njit 
def numba_integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s

#now the decorated function will run entirely without 
#the involvement of the Python interpreter.

In [9]:
jittedtime = %timeit -o -n 10 numba_integrate(a, b, n)

The slowest run took 27.00 times longer than the fastest. This could mean that an intermediate result is being cached.
5.22 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
np.testing.assert_almost_equal(numba_integrate(a, b, n), integrate(a, b, n))

In [11]:
plaintime.best/jittedtime.best

122.55723884423224

<font size = 6>Why is there a slow run? </font>

Numba has to compile your function for the argument types given before it executes the machine code version of your function. This takes time. 

However, once the compilation has taken place Numba caches the machine code version of your function for the particular types of datatypes presented. 

If it is called again, it can reuse the cached version, if it sees the same datatypes, instead of having to compile again.

In [22]:
@numba.njit
def numba_integrate(a, b, n):
    s = 0.0
    dx = (b - a) / n
    for i in range(n):
        x = a + (i + 0.5) * dx
        y = x ** 4 - 3 * x
        s += y * dx
    return s          

# COMPILATION TIME!
start = time.perf_counter()
numba_integrate(a, b, n)
end = time.perf_counter()
print(f"Elapsed (with compilation) = {end - start:.3f} s")

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
for i in range(5):
    start = time.perf_counter()
    numba_integrate(a, b, n)
    end = time.perf_counter()
    print(f"Elapsed (cached)           = {end - start:.3f} s")

#Original function
start = time.perf_counter()
numba_integrate.py_func(a, b, n)
end = time.perf_counter()
print(f"Elapsed (python code)      = {end - start:.3f} s")

Elapsed (with compilation) = 0.058 s
Elapsed (cached)           = 0.001 s
Elapsed (cached)           = 0.001 s
Elapsed (cached)           = 0.001 s
Elapsed (cached)           = 0.001 s
Elapsed (cached)           = 0.001 s
Elapsed (python code)      = 0.133 s


In [28]:
numba_integrate.signatures

[(int64, int64, int64), (float64, float64, int64), (float64, float64, float64)]

In [27]:
numba_integrate(0.0, 2.0, 1_000_000.)

0.3999999999947188

## What can you use numba on?

If your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice.

* Functions with loops
* Functions with mathematical operations
* Functions using numpy

***OK***: tuples, strings

***Not good***: objects, Python lists, python dicts

<font size=8> **Hands on exercise** <font>

Function ```estimate_pi``` calculates the value of $pi$

Notice that the greater the value of n, the better is the estimate of $pi$

You task is to:
+ Use time or timeit to benchmark the speed up in execution of the function for different *n* with and without numba
+ Plot the difference in time for different *n* with and without numba 

**(Ensure that the numba function is compiled before timing the function)**

In [34]:
#This function takes n as input which is the number of total points within the square
#It then estimates how many of those points are within the circle
# value of pi = 4 * numberofpointsincircle/numberofpointsinsquare

def estimate_pi(n):
    np.random.seed(1234)
    count_inside_circle = 0
    for _ in range(n): # Numba likes loops and numpy functions
        x, y  = [np.random.uniform(-1, 1),
                 np.random.uniform(-1, 1)]
        
        if x**2 + y**2 <= 1:
            count_inside_circle += 1
    
    pi_estimate = 4 * count_inside_circle / n #Numba likes math
    return pi_estimate

In [35]:
%timeit estimate_pi(1_000_000)

3.24 s ± 54.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
n = [10, 1000, 10000, 100_000, 1_000_000]
for i in n:
    pi_estimate = estimate_pi(i)
    print("Number of runs %d, Estimated Pi: %0.2f" %(i, pi_estimate))

Number of runs 10, Estimated Pi: 3.60
Number of runs 1000, Estimated Pi: 3.06
Number of runs 10000, Estimated Pi: 3.12
Number of runs 100000, Estimated Pi: 3.14
Number of runs 1000000, Estimated Pi: 3.14


In [ ]:
## Your code here

In [38]:
@numba.njit
def estimate_pi(n):
    np.random.seed(1234)
    count_inside_circle = 0
    for _ in range(n): # Numba likes loops and numpy functions
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        
        if x**2 + y**2 <= 1:
            count_inside_circle += 1
    
    pi_estimate = 4 * count_inside_circle / n #Numba likes math
    return pi_estimate

In [33]:
n = [10, 1000, 10000, 100_000, 1_000_000]
for i in n:
    pi_estimate = estimate_pi(i)
    print("Number of runs %d, Estimated Pi: %0.2f" %(i, pi_estimate))

Number of runs 10, Estimated Pi: 3.60
Number of runs 1000, Estimated Pi: 3.06
Number of runs 10000, Estimated Pi: 3.12
Number of runs 100000, Estimated Pi: 3.14
Number of runs 1000000, Estimated Pi: 3.14


In [39]:
%timeit estimate_pi(1_000_000)

8.23 ms ± 39.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## What does numba *not* do? : Choose wisely!
* Magically make individual numpy functions faster, or speed up code that just has nice array operations
* Convert aribitrary python data types
* Translate entire programs
* Compile third party libraries
* Numba cannot help speed up algorithms that are not primarily numerical

> @njit stands for @jit(nopython=True) and it means that numba should throw an exception instead of falling back to the python version, in cases where it cannot generate the optimized version.